In [4]:
import Ipynb_importer
from losses import *
from constraints import get_initializer
from base_model import KnowledgeGraphEmbeddingModel

       
        
class Cnmsb(KnowledgeGraphEmbeddingModel):
    """
    The embedding model (Cnmsb)
    """

    def __init__(self, em_size=100, batch_size=128, nb_epochs=100, initialiser="xavier_uniform", nb_negs=2,
                 optimiser="amsgrad", loss="default", lr=0.01, nb_ents=0, nb_rels=0, reg_wt=0.01, seed=1234, verbose=1,
                 log_interval=5):
        """ Initialise new instance of the ComplEx model
        Parameters
        ----------
        em_size: int
            embedding vector size
        batch_size: int
            batch size
        nb_epochs: int
            number of epoch i.e training iterations
        initialiser: str
            initialiser name e.g. xavier_uniform or he_normal
        nb_negs: int
            number of negative instance per each positive training instance
        optimiser: str
            optimiser name
        lr: float
            optimiser learning rate
        nb_ents: int
            total number of knowledge graph entities
        nb_rels: int
            total number of knowledge graph relations
        reg_wt: float
            regularisation parameter weight
        seed: int
            random seed
        verbose: int
            verbosity level. options are {0, 1, 2}
        log_interval: int
            the number of epochs to wait until reporting the next training loss. (loss logging frequency)
        """
        super().__init__(em_size=em_size, batch_size=batch_size, nb_epochs=nb_epochs, initialiser=initialiser,
                         nb_negs=nb_negs, optimiser=optimiser, lr=lr, loss=loss, nb_ents=nb_ents, nb_rels=nb_rels,
                         reg_wt=reg_wt, seed=seed, verbose=verbose, log_interval=log_interval)

    def init_embeddings(self):
        """ Initialise the embeddings for both entities and relations.
        Returns
        -------
        tf.Variable
            _embeddings of knowledge graph entities
        tf.Variable
            _embeddings of knowledge graph relations
        """
        # get initialiser variable and initialise tensorflow variables for each component _embeddings
        var_init = get_initializer(self.initialiser, self.seed)
        em_ents = tf.compat.v1.get_variable("em_ents", shape=[self.nb_ents+1, self.em_size*2], initializer=var_init)
        em_rels = tf.compat.v1.get_variable("em_rels", shape=[self.nb_rels+1, self.em_size*2], initializer=var_init)

        # add component embedding to the embedding vars dictionary
        self._embeddings["ents"] = em_ents
        self._embeddings["rels"] = em_rels

        return em_ents, em_rels

    def score_triples(self, sub_em, rel_em, obj_em):
        """ Compute scores for a set of triples given their component _embeddings
        Parameters
        ----------
        sub_em: tf.tensor
            Embeddings of the subject entities
        rel_em: tf.tensor
            Embeddings of the relations
        obj_em: tf.tensor
            Embeddings of the object entities
        Returns
        -------
        tf.tensor
            model scores for the original triples of the given _embeddings
        """
        em_interactions = sub_em * rel_em * obj_em
        scores = tf.reduce_sum(em_interactions, axis=1)
        return scores

    
    def compute_loss(self, scores, *args, **kwargs):
        """ Compute training loss using the loss function
        Parameters
        ----------
        scores: tf.Tenor
            scores tensor
        args: list
            Non-Key arguments
        kwargs: dict
            Key arguments
        Returns
        -------
        tf.float32
            model loss value
        """
        if self.loss == "default":
            pos_scores, neg_scores = tf.split(scores, num_or_size_splits=2)
            targets = tf.concat((tf.ones(tf.shape(pos_scores)), -1 * tf.ones(tf.shape(neg_scores))), axis=0)
            return pointwise_logistic_loss(scores, targets=targets, reduction_type="avg")
        else:
            return compute_kge_loss(scores, self.loss, reduction_type="avg")
        